In [1]:
import json
import base64
import sqlite3
import win32crypt
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import requests
import re
from requests_toolbelt import MultipartEncoder

In [2]:
cd D:\USER\OneDrive\jupyter notebook\網路爬蟲

D:\USER\OneDrive\jupyter notebook\網路爬蟲


In [3]:
def GetString(LocalState):
    with open(LocalState,'r',encoding='utf-8') as f:
        s=json.load(f)['os_crypt']['encrypted_key']
    return s

def pull_the_key(base64_encrypted_key):
    encrypted_key_with_header=base64.b64decode(base64_encrypted_key)
    encrypted_key=encrypted_key_with_header[5:]
    key=win32crypt.CryptUnprotectData(encrypted_key,None,None,None,0)[1]
    return key

def DecryptString(key,data):
    nonce,cipherbytes=data[3:15],data[15:]
    aesgcm=AESGCM(key)
    plainbytes=aesgcm.decrypt(nonce,cipherbytes,None)
    plaintext=plainbytes.decode('utf-8')
    return plaintext

def test_cookies(domains):
    LocalState = r'C:\Users\User\AppData\Local\Google\Chrome\User Data\Local State' #密钥文件
    Cookies = r'C:\Users\User\AppData\Local\Google\Chrome\User Data\Default\Network\Cookies' #cookie文件
    
    con = sqlite3.connect(Cookies)
    if type(domains) == str:
        sql_command = 'select host_key,name,encrypted_value from cookies where host_key like "%' + domains + '%"'
    if type(domains) == list:
        sql_command = 'select host_key,name,encrypted_value from cookies where host_key like "%' + domains[0] + '%"'
        for domain in domains[1:]:
            sql_command = sql_command + 'or host_key like "%' + domain + '%"'
    
    res_domain_name = con.execute(sql_command).fetchall()
        
    key = pull_the_key(GetString(LocalState))        
    con.close()

#    列表回傳  
    datas={}
    for i in res_domain_name:
        if i[1] == "":
            continue
        datas[i[1]]=DecryptString(key, i[2])
    return datas

In [4]:
#填入欲開啟的檔案名稱、檔案位置與檔案類型
file_payload = {'fileUpload': ("456.jpg",open("D:\\456.jpg",'rb'), "image/jpg")}

# 直接利用 MultipartEncoder 將圖片檔案轉成「表單資料 ( form data )」：可用于 multipart/form-data 上傳的數據
my_data = MultipartEncoder(file_payload)

# 自動生成 Content-Type 類型和隨機碼
my_headers = {}
my_headers['Content-Type'] = my_data.content_type

# 查詢參數，複製網站上的「查詢字串參數」即可，推測這個參數是與 Cookies 有關，後續可使用下方 Cookies 來填入
my_params = {'_cachebust': '1641745758', 'js_session': '%7B%22apikey%22:%22ADNupMnWyR7kCWRvm76Laz%22,%22mimetypes%22:%5B%22application%2F*%22,%22image%2F*%22,%22audio%2F*%22,%22video%2F*%22%5D,%22persist%22:false,%22version%22:%22v2%22,%22storeLocation%22:%22S3%22,%22storePath%22:null,%22storeContainer%22:null,%22storeAccess%22:%22private%22,%22storeRegion%22:null,%22external%22:true%7D'}

# Cookies
image_cookies = test_cookies('filepicker')

# 將相關參數加入 POST 請求中，包含「標頭 ( headers)」、「查詢字串參數」、Cookies、「表單資料 ( form data )」
r = requests.post('https://www.filepicker.io/api/upload/',headers = my_headers, params = my_params, cookies = image_cookies, data = my_data)

# 確認 API 的呼叫狀況
if r.ok:
    print("API 請求成功")
else:
    print("API 請求失敗 QQ")
    
# 取得圖片雲端網址，json.loads 可將請求發送成功後，回傳的資料轉成 dict 格式 (字典檔)
image_url = json.loads(r.text)['data'][0]['url']

# 透過正則表達式組合成圖片的完整網址
image_url = 'https://cdn.fs.teachablecdn.com/' + re.match(r'https://www.filepicker.io/api/file/(.*)',image_url)[1]

API 請求成功


In [5]:
def get_course_no(rq_response,course_name):
    courses_info = json.loads(rq_response.text)
    for course_info in courses_info['data']:
        if course_info['attributes']['name'] == course_name:
            return str(course_info['attributes']['id'])
        else:
            return

In [6]:
teachable_cookies = test_cookies(['teachable.com','teciram904-s-school.teachab'])
r = requests.get('https://teciram904-s-school.teachable.com/api/v2/courses?page=1&per=20', cookies = teachable_cookies)
course_no = get_course_no(r,'teciram904')
if type(course_no) is type(None):
    print('查無此課程名稱')

In [7]:
r = requests.get('https://teciram904-s-school.teachable.com/api/v2/courses/' + course_no+'/pages?include[]=draft', cookies = teachable_cookies)
page_info = json.loads(r.text)
page_no = page_info['included'][0]['id']

In [8]:
def get_block_no(rq_response,block_name = "banner"): # banner | image_with_text | text | text_with_image_grid ...etc
    blocks_info = json.loads(rq_response.text)
    for block_info in blocks_info['blocks']:
        if re.match(re.compile(".*" + block_name + ".*"),block_info['partial']):
            return str(block_info['id'])
        else:
            return

In [9]:
r = requests.get('https://teciram904-s-school.teachable.com/api/v1/pages/' + page_no, cookies = teachable_cookies)
block_no = get_block_no(r)

In [10]:
# 取得 csrf token
r = requests.get('https://teciram904-s-school.teachable.com/admin/site/pages/'+page_no+'/edit', cookies = teachable_cookies)
# 為了避免抓錯，利用 findall 找出所有相符的模式，測試後取第一個即可
csrf_token = re.findall(r'csrf_token\"\:\"(.*?)\"\,',r.text.strip())[0]

teachable_cookies = test_cookies(['teachable.com','teciram904-s-school.teachab'])

In [11]:
r.url

'https://teciram904-s-school.teachable.com/admin/site/pages/9761403/edit'

In [13]:
csrf_token

'dcIzWgftknk8k3XFTPJ/ZD9B9YgtNRxCsNFU2K91Fwzig1I0qGhfaqyefp/FIGpQiLTKzf/G84nfBIJo96JXUw=='

In [666]:
body={}
body['id']=block_no
body['data']={}
body['data']['image_url']=image_url
body['data']['alt_text']=''
body['data']['heading_text']='<h1>Example Banner</h1>'
body['data']['subheading_text']='<p><strong>Showcase your course or coaching, an announcement, or big feature. Use an eye-catching subheader to give your audience more context.</strong></p>'
body['data']['button_partial']='external_link'
body['data']['button_action_url']='#'
body['data']['button_text']='[Set a button action]'
body['toggles']={}
body['toggles']['show_button']=True

In [667]:
# 替標頭加上 csrf token
csrf_headers = {}
csrf_headers['x-csrf-token'] = csrf_token
csrf_headers['Content-Type'] = "application/json"

r = requests.put('https://teciram904-s-school.teachable.com/api/v1/pages/' + page_no + '/blocks/' + block_no, headers = csrf_headers, cookies = teachable_cookies, data = json.dumps(body))
if json.loads(r.text)['data']['image_url']==image_url:
    print('good')

good


In [668]:
r = requests.post('https://teciram904-s-school.teachable.com/api/v2/pages/'+ str(page_info['data'][0]['attributes']['id']) +'/publish', headers = csrf_headers, cookies = teachable_cookies)
if r.status_code == 204:
    print('good')

good


In [8]:
# teachable_cookies = test_cookies(['teachable.com','teciram904-s-school.teachab'])

# 設定 該頁面ID
page_no="9761403"

# 設定 該頁面該區塊的 ID
block_no="84107927"

# 新的頁面資料~~。除了 image_url 這個屬性之外，其他都可以複製網站上的 API 的資料
body={}
body['id']=block_no
body['data']={}
body['data']['image_url']=image_url
body['data']['alt_text']=''
body['data']['heading_text']='<h1>Example Banner</h1>'
body['data']['subheading_text']='<p><strong>Showcase your course or coaching, an announcement, or big feature. Use an eye-catching subheader to give your audience more context.</strong></p>'
body['data']['button_partial']='external_link'
body['data']['button_action_url']='#'
body['data']['button_text']='[Set a button action]'
body['toggles']={}
body['toggles']['show_button']=True

# 替標頭加上 csrf token，熱騰騰的 csrf token  rrrrrrrr
csrf_headers = {}
csrf_headers['x-csrf-token'] = csrf_token
csrf_headers['Content-Type'] = "application/json"

# 將相關參數加入 put 請求中，包含 Cookies、新的頁面資料 (json格式)
r = requests.put('https://teciram904-s-school.teachable.com/api/v1/pages/' + page_no + '/blocks/' + block_no, headers = csrf_headers, cookies = teachable_cookies, data = json.dumps(body))

# 確認 API 的呼叫狀況
if r.ok:
    print("API 請求成功")
else:
    print("API 請求失敗 QQ")

In [11]:
r.status_code

422